In [3]:
import ghsl_time_consistent_cluster as tcc
import config as config

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. ETL Pipeline
Here we extract the data from the pop and smod rasters and copy it to postigs. 

In [3]:
tcc.load_ghsl_rasters()

Running bash script with arguments:
CREATE EXTENSION
CREATE EXTENSION
Loading data for year 1975...
BEGIN
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
COPY 1000
COPY 1000
COPY 1000
COPY 171
COPY 1000
COPY 1000
COPY 556
COPY 648
COPY 162
COPY 45
addrasterconstraints
----------------------
t
(1 row)

addrasterconstraints
----------------------
t
(1 row)

addrasterconstraints
----------------------
t
(1 row)

addrasterconstraints
----------------------
t
(1 row)

addrasterconstraints
----------------------
t
(1 row)

addoverviewconstraints
------------------------
t
(1 row)

addoverviewconstraints
------------------------
t
(1 row)

addoverviewconstraints
------------------------
t
(1 row)

addoverviewconstraints
------------------------
t
(1 row)

COMMIT
BEGIN
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
COPY 1000
COPY 1000
COPY 1000
COPY 1000
COPY 1000
COPY 1000
COPY 1000
COPY 1000
COPY 2
COPY 1000
COPY 1000
COPY 556
COPY 648
COPY 162
COPY 45
addr

# 2. Clusters
Here we build clusters, the central unit of analysis of this project. Clusters are defined build as follows:
- 2.0 Extract urban areas from the smod raster and then compute population zonal statistics over those urban areas using the pop raster

In [4]:
tcc.create_cluster()

2024-07-23 16:20:16,162 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-07-23 16:20:16,162 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-23 16:20:16,163 INFO sqlalchemy.engine.Engine select current_schema()
2024-07-23 16:20:16,163 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-23 16:20:16,164 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-07-23 16:20:16,164 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-23 16:20:16,165 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-23 16:20:16,168 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS "cluster_1975";

-- Create a binary raster from the smod raster with only urban and non-urban classes
CREATE TEMPORARY TABLE smod_binary ON COMMIT DROP AS
SELECT ST_Reclass(smod_1975.rast, 1,  '[0-21]:0, (21-30]:1', '1BB', nodataval := 0) AS rast
FROM smod_1975;

-- Create a temporary cluster geometry table with DBSCAN
CREATE TEMPORARY TABLE cluster_geom ON COMMIT DROP AS
WITH urban_pixels AS (
    SELE

# 3. Time consistent clusters
Here we build time consistent clusters i.e., clusters having a consistent id across time. We do this by:

- 3.1 Create a table with all clusters in all years. Create the "cluster intersection matching": a matching where two clusters (from different years) are matched if they intersect.
- 3.2 Create a crosswalk table between cluster ids and time consistent cluster ids. Time consistent clusters are defined as connected components of bipartite graph representing the cluster intersection matching. 
- 3.3 Create a table with time consistent clusters geometry. Create a table with the time consistent cluster population across years.

In [5]:
tcc.create_multiyear_tables_and_cluster_intersection_matching()

2024-07-24 16:07:05,127 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-07-24 16:07:05,127 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-24 16:07:05,128 INFO sqlalchemy.engine.Engine select current_schema()
2024-07-24 16:07:05,128 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-24 16:07:05,129 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-07-24 16:07:05,129 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-24 16:07:05,130 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-24 16:07:05,131 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS multiyear_cluster
2024-07-24 16:07:05,131 INFO sqlalchemy.engine.Engine [generated in 0.00022s] {}
2024-07-24 16:07:05,135 INFO sqlalchemy.engine.Engine CREATE TABLE multiyear_cluster AS (SELECT 1975 as year, cluster_id, population, geom FROM cluster_1975 UNION ALL SELECT 1980 as year, cluster_id, population, geom FROM cluster_1980 UNION ALL SELECT 1985 as year, cluster_id, population, geom FROM cluster

In [6]:
tcc.create_crosswalk_cluster_uid_to_cluster_id()

2024-07-24 16:07:42,739 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-24 16:07:42,739 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2024-07-24 16:07:42,740 INFO sqlalchemy.engine.Engine [generated in 0.00085s] {'table_name': 'SELECT * FROM cluster_intersection_matching', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2024-07-24 16:07:42,741 INFO sqlalchemy.engine.Engine SELECT * FROM cluster_intersection_matching
2024-07-24 16:07:42,742 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-24 16:07:53,622 INFO sq

In [8]:
tcc.create_time_consistent_cluster_geometry_pre_geocoding()

2024-07-24 16:23:04,263 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-24 16:23:04,265 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS "time_consistent_cluster_geometry_pre_geocoding";

-- Create the time consistent cluster geometry table
CREATE TABLE "time_consistent_cluster_geometry_pre_geocoding" AS
WITH multiyear_cluster_with_uid AS (
    SELECT m.cluster_uid, c.year, c.cluster_id, geom
    FROM "crosswalk_cluster_uid_to_cluster_id" m JOIN "multiyear_cluster" c
    ON m.cluster_id = c.cluster_id AND m.year = c.year
)
SELECT cluster_uid, ST_Union(geom) AS geom
FROM multiyear_cluster_with_uid
GROUP BY cluster_uid;

CREATE INDEX ON "time_consistent_cluster_geometry_pre_geocoding" USING GIST (geom);
2024-07-24 16:23:04,265 INFO sqlalchemy.engine.Engine [generated in 0.00034s] {}
2024-07-24 16:23:49,908 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
tcc.create_time_consistent_cluster_pre_geocoding()

2024-07-24 19:30:16,179 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-24 19:30:16,180 INFO sqlalchemy.engine.Engine DROP TABLE time_consistent_cluster_1975;

CREATE TABLE time_consistent_cluster_1975 AS
WITH zonal_stats AS (
    SELECT cluster_uid, (St_SummaryStats(St_Union(ST_Clip(rast, 1, geom, true)))).*
    FROM time_consistent_cluster_geometry_pre_geocoding, pop_1975
    WHERE St_Intersects(rast,geom)
    GROUP BY cluster_uid
)
SELECT cluster_uid, sum AS population
FROM zonal_stats;
2024-07-24 19:30:16,181 INFO sqlalchemy.engine.Engine [generated in 0.00032s] {}
2024-07-24 19:31:20,747 INFO sqlalchemy.engine.Engine DROP TABLE time_consistent_cluster_1980;

CREATE TABLE time_consistent_cluster_1980 AS
WITH zonal_stats AS (
    SELECT cluster_uid, (St_SummaryStats(St_Union(ST_Clip(rast, 1, geom, true)))).*
    FROM time_consistent_cluster_geometry_pre_geocoding, pop_1980
    WHERE St_Intersects(rast,geom)
    GROUP BY cluster_uid
)
SELECT cluster_uid, sum AS population
FROM

# 4. Country Geocoding
Here we load the country boarder data and assign to each cluster, year pair the country that the cluster belongs into the given year. 
- 4.1 Load the country boarder data
- 4.2 Assign the country to each cluster, year pair. Here we make a couple of choices thar are worth noting:
    - Border clusters are assigned to the country with the largest area of intersection
    - Small islands that intersect no country are removed. These are typically islands so small that they are not counted in the country border dataset.

In [16]:
tcc.load_country_borders()

2024-07-24 20:25:20,711 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-24 20:25:20,711 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS country_borders
2024-07-24 20:25:20,711 INFO sqlalchemy.engine.Engine [generated in 0.00020s] {}
2024-07-24 20:25:20,716 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS crosswalk_cshape_to_world_bank_codes
2024-07-24 20:25:20,716 INFO sqlalchemy.engine.Engine [generated in 0.00035s] {}
2024-07-24 20:25:20,717 INFO sqlalchemy.engine.Engine COMMIT
Running bash script with arguments:
CREATE TABLE
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INS

In [17]:
tcc.geocode_cluster_with_country()

2024-07-24 20:25:23,337 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-24 20:25:23,340 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS time_consistent_cluster;
DROP TABLE IF EXISTS time_consistent_cluster_geometry;

-- Temporary table of transformed country borders
CREATE TEMPORARY TABLE temp_country_geom_transformed ON COMMIT DROP AS
SELECT gwcode, gwsyear, gweyear, ST_transform(the_geom, 54009) AS geom
FROM country_borders;

CREATE INDEX ON temp_country_geom_transformed USING GIST (geom);

-- Temporary table of cluster-country matching
CREATE TEMPORARY TABLE temp_cluster_country_matching ON COMMIT DROP AS
SELECT cluster_uid, year, gwcode, gwsyear, gweyear
FROM (time_consistent_cluster_pre_geocoding JOIN time_consistent_cluster_geometry_pre_geocoding USING (cluster_uid)) cluster
JOIN temp_country_geom_transformed country ON st_intersects(cluster.geom, country.geom)
WHERE (gwsyear < year AND year <= gweyear) OR (year = 2020 AND gweyear = 2019);

-- Temporary table of cluster